In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch


In [ ]:
device="cuda"
torch_dtype = None
BASE="Qwen/Qwen2.5-3B-Instruct"
ADAPTER = "abdelalem33/coder"

model = AutoModelForCausalLM.from_pretrained(
    BASE,
    device_map="auto",
    torch_dtype = torch_dtype
).to(device)

model_with_adaptor = PeftModel.from_pretrained(
    model,
    ADAPTER,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(BASE)


In [ ]:
#to see  model arch with lora
model.eval()

In [ ]:
def generate_resp(message):
    message=[{"role":"system","content":"write just python code"},{"role": "user", "content": messages}]

    text = tokenizer.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer([text],
                             return_tensors="pt",
                             padding=True).to(device)

    generated_ids = model_with_adaptor.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,

    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response


In [ ]:
msg="convert list [1,2,5] to one integer"
print(generate_resp(msg))